Загрузим нужные библиотеки

In [1]:
! python -m spacy download ru_core_news_sm

                                              0.0/15.3 MB ? eta -:--:--
                                              0.0/15.3 MB 1.3 MB/s eta 0:00:12
                                              0.1/15.3 MB 1.1 MB/s eta 0:00:15
                                             0.1/15.3 MB 983.0 kB/s eta 0:00:16
                                              0.2/15.3 MB 1.4 MB/s eta 0:00:11
     -                                        0.4/15.3 MB 1.5 MB/s eta 0:00:10
     -                                        0.6/15.3 MB 1.8 MB/s eta 0:00:09
     -                                        0.7/15.3 MB 1.9 MB/s eta 0:00:08
     --                                       0.8/15.3 MB 1.8 MB/s eta 0:00:08
     --                                       1.1/15.3 MB 2.0 MB/s eta 0:00:08
     ---                                      1.2/15.3 MB 2.0 MB/s eta 0:00:08
     ---                                      1.3/15.3 MB 2.0 MB/s eta 0:00:07
     ---                                      1.4/15.3 MB 

In [2]:
import pandas as pd
import unicodedata
import re
import spacy
import json
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import json
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess

# Подготовка данных

Загрузим данные

In [3]:
data = pd.read_csv('csv\\full_df.csv', sep=';')
data.head(3)
data = data.drop(columns='Unnamed: 0', axis=1)

Отфильтруем тексты по количеству символов. Оставим только 100+.

In [4]:
data['text_len'] = data.loc[:, 'text'].apply(lambda x: len(x))
data_filtered_by_text_len = data.query('text_len > 100')
print(f'Количество постов с 100+ символами: {data_filtered_by_text_len.shape[0]}')
data_filtered_by_text_len.head(3)

Количество постов с 100+ символами: 8556


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
2,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",17.0,6.0,0.0,117


Оставим только тексты содержащие кириллицу

In [5]:
data_cyrillic = data_filtered_by_text_len[data_filtered_by_text_len['text'].apply(lambda x: re.match(r'[А-Яа-я]+', x) is not None)]

print(f'Текстов на кириллице: {data_cyrillic.shape[0]}')
data_cyrillic.head(3)

Текстов на кириллице: 1960


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
2,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",17.0,6.0,0.0,117


Удалим дубликаты текстов

In [6]:
data_dd = data_cyrillic.drop_duplicates('text')
print(f'Осталось {data_dd.shape[0]} строк')
data_dd.head(3)

Осталось 1862 строк


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
6,Talent Acquisition Manager | Recruitment Lead ...,https://www.linkedin.com/in/dariaivanova,С трепетом готов представить свой продукт. Реф...,8.0,0.0,0.0,841


# Подготовка к моделированию

In [7]:
texts = data_dd["text"].tolist()

Проведем лемматизацию

In [8]:
if os.path.exists('lemmas.json'):
    with open("lemmas.json") as f:
        data_lemmatized = json.load(f)['lemmas']
else:
    model = spacy.load('ru_core_news_sm', disable=['ner', 'parser'])
    data_lemmatized = []
    for doc in model.pipe(texts, disable=["tagger", "parser"]):
        data_lemmatized.append([token.lemma_ for token in doc])

    with open("lemmas.json", "w") as fid:
        json.dump({"lemmas": data_lemmatized}, fid)

print(data_lemmatized[:1])

[['всем', 'привет!выпустил', 'свой', 'первый', 'статья', 'на', 'хабр!https://lnkd.in', '/', 'dt9n6d7bстатья', 'про', 'история', 'и', 'технология', 'разработка', 'игра', 'https://guess-word.com', 'и', 'как', 'мы', 'создать', 'игра', 'с', 'элемент', 'машинный', 'обучение', 'и', 'выйти', 'в', 'ноль', 'за', '2', 'месяцапри', '\xa0', 'внимательный', 'прочтение', 'вы', 'даже', 'смочь', 'запустить', 'первый', 'версия', 'игра', '!']]


Очистим тексты от ненужных символов

In [9]:
word_pattern = re.compile("^[а-я]*$")

def remove_symbols(doc):
    return [token for token in doc if word_pattern.match(token)]

data_words = list(map(remove_symbols, data_lemmatized))

Загрузим русские стоп-слова

In [10]:
stop_words = stopwords.words('russian')
stop_words

Создадим биграммы и триграммы

In [11]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_lemmatized], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[data_words[0]]])

['всем', 'свой', 'первый', 'статья', 'на', 'про', 'история', 'и', 'технология', 'разработка', 'игра', 'и', 'как', 'мы', 'создать', 'игра', 'с', 'элемент', 'машинный_обучение', 'и', 'выйти', 'в', 'ноль', 'за', 'месяцапри', 'внимательный', 'прочтение', 'вы', 'даже', 'смочь', 'запустить', 'первый', 'версия', 'игра']


Определим функции для удаления стоп-слов, создания биграмм и триграмм

In [12]:
def remove_stopwords(texts):
    return [[word for word in doc if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


Применим функции для удаления стоп-слов и создания биграмм

In [13]:
data_words_nostops = remove_stopwords(data_words)

data_words_bigrams = make_bigrams(data_words_nostops)

In [14]:
id2word = corpora.Dictionary(data_words_bigrams)

texts = data_words_bigrams

corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 3), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)]]


In [15]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('версия', 1),
  ('внимательный', 1),
  ('всем', 1),
  ('выйти', 1),
  ('запустить', 1),
  ('игра', 3),
  ('история', 1),
  ('машинный_обучение', 1),
  ('месяцапри', 1),
  ('ноль', 1),
  ('первый', 2),
  ('прочтение', 1),
  ('разработка', 1),
  ('свой', 1),
  ('смочь', 1),
  ('создать', 1),
  ('статья', 1),
  ('технология', 1),
  ('элемент', 1)]]

In [16]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=20, 
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha='auto',
    per_word_topics=True
    )

In [17]:
lda_model.print_topics()

[(0,
  '0.052*"профиль" + 0.050*"продажа" + 0.040*"целое" + 0.032*"ребята" + 0.032*"надеяться" + 0.030*"отзыв" + 0.027*"котором" + 0.022*"дом" + 0.016*"участвовать" + 0.015*"голос"'),
 (1,
  '0.062*"дизайнер" + 0.055*"встреча" + 0.046*"создать" + 0.040*"рекрутер" + 0.026*"предоставлять" + 0.026*"достигнуть" + 0.023*"популярный" + 0.019*"собирать" + 0.016*"базовый" + 0.016*"объяснить"'),
 (2,
  '0.049*"мной" + 0.039*"конкурент" + 0.035*"кстати" + 0.031*"цена" + 0.030*"разобраться" + 0.018*"единственный" + 0.011*"александр" + 0.010*"туда" + 0.010*"соцсеть" + 0.009*"чудо"'),
 (3,
  '0.057*"которой" + 0.050*"инструмент" + 0.032*"действие" + 0.031*"суть" + 0.031*"программист" + 0.031*"подробность" + 0.021*"октябрь" + 0.021*"метод" + 0.020*"площадка" + 0.016*"кофе"'),
 (4,
  '0.059*"исследование" + 0.043*"стратегия" + 0.033*"коммуникация" + 0.020*"опубликовать" + 0.005*"проблемный" + 0.000*"виноватый" + 0.000*"робототехника" + 0.000*"выбор" + 0.000*"ресурс" + 0.000*"просить"'),
 (5,
  '0.053

In [18]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -16.143723320522806

Coherence Score:  0.45406890945420464


In [19]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15     0.426242  0.066590       1        1  47.676456
19     0.342046 -0.312268       2        1  18.723723
13     0.301914  0.266339       3        1  15.346374
10     0.004837 -0.020162       4        1   3.420430
14    -0.007827 -0.029070       5        1   3.151337
9     -0.061425  0.001156       6        1   1.446724
17    -0.060004  0.001063       7        1   1.426046
12    -0.056942 -0.000523       8        1   1.352935
6     -0.055073  0.002379       9        1   1.242411
3     -0.065723  0.001425      10        1   1.186451
1     -0.069985  0.001722      11        1   1.079449
0     -0.068657  0.001627      12        1   1.018894
8     -0.074689  0.001996      13        1   0.801390
2     -0.077990  0.002249      14        1   0.486034
18    -0.074450  0.002718      15        1   0.441528
5     -0.079850  0.002437      16        1   0.399349
4     -0.080277  0.002517      17        1   0.263644
7     -0.080424  0.002519      18        1   0.244084
16    -0.080820  0.002602      19        1   0.189413
11    -0.080903  0.002684      20        1   0.103326, topic_info=            Term         Freq        Total Category  logprob  loglift
46      компания  1065.000000  1065.000000  Default  30.0000  30.0000
99           год   905.000000   905.000000  Default  29.0000  29.0000
90           это  2336.000000  2336.000000  Default  28.0000  28.0000
105       задача   443.000000   443.000000  Default  27.0000  27.0000
240       работа  1925.000000  1925.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
4524      сессия     0.006553     0.853941  Topic20  -9.7113   2.0052
1886  английский     0.006554     0.853975  Topic20  -9.7113   2.0051
5474      аспект     0.006553     0.853938  Topic20  -9.7113   2.0052
2612         мир     0.006554     0.854061  Topic20  -9.7113   2.0050
4028      ресурс     0.006554     0.853994  Topic20  -9.7113   2.0051

[857 rows x 6 columns], token_table=      Topic      Freq               Term
term                                    
1572      8  0.914409         автомобиль
1076      1  0.986448              автор
1598     11  0.459785  администрирование
1793     15  0.959998               айти
694       2  0.993151            активно
...     ...       ...                ...
1765     15  0.704232             эпизод
90        1  0.994373                это
90        3  0.005565                это
1402      7  0.996311           являться
223       2  0.996726               язык

[514 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 20, 14, 11, 15, 10, 18, 13, 7, 4, 2, 1, 9, 3, 19, 6, 5, 8, 17, 12])

In [20]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(
            corpus=corpus,
            id2word=id2word,
            num_topics=num_topics, 
            random_state=100,
            update_every=1,
            chunksize=100,
            passes=10,
            alpha='auto',
            per_word_topics=True
            )
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [21]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_words_bigrams, start=2, limit=50, step=6)

In [ ]:
limit=50
start=2
step=6
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()